## COVID-19 Open Research Project - CS431 Final Project

## Set up data and environment

In [0]:
from pyspark import SparkContext, SparkConf

from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession, Row
from pyspark.ml import Pipeline

In [0]:
import glob

In [0]:
# File location and type
file_location = "/FileStore/tables/COR/"
file_type = "json"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.json(file_location, multiLine=True)

df.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 abstract| back_matter| bib_entries| body_text| metadata| paper_id| ref_entries|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
[[[], [], Abstrac...|[[[], [], Conflic...|[[[[A, Ablasser, ...|[[[[219, BIBREF81...|[[[[Norwegian Uni...|1c66ec8950cbd1e0f...|[[, Meta-analysis...|
[[[], [], Abstrac...|[[[], [], Acknowl...|[[[[T, Dörner, []...|[[[], [], , Keywo...|[[[[Carnegie Mell...|2c006d09b6fccc527...|[[, generated a d...|
[[[], [], Abstrac...|[[[], [], acknowl...|[[[[E, Dong, [], ...|[[[[76, BIBREF3, ...|[[[[Indiana Unive...|2b90a9a3d33a1babe...|[[, . (See Metzca...|
[[[], [], Abstrac...|[[[], [], ACKNOWL...|[[[[A, Varki, [],...|[[[], [], , 3 dif...|[[[[Cornell Unive...|1a791c5c6543aca93...|[[, 65 structural...|
[[[], [], Abstrac...|[[[], [[610,, 1, ...|[, [[[, Lac Chapm...|[[[[177,, 175, 51...|[[[[,,], , L, , [...|1d882d90109fb4496...|[[, 45 since 2011...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows

In [0]:
df.count()

Out[46]: 101

## Preprocess Data

In [0]:
df.printSchema()

root
-- abstract: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- cite_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- ref_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- section: string (nullable = true)
 | |-- text: string (nullable = true)
-- back_matter: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- cite_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- ref_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- section: string (nullable = true)
 | |-- text: string (nullable = true)
-- bib_entries: struct (nullable = true)
 |-- BIBREF0: struct (nullable = true)
 | |-- authors: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- first: string (nullable = true)
 | | | |-- last: string (nullable = true)
 | | | |-- middle: array (nullable = true)
 | | | | |-- element: string (containsNull = true)
 | | | |-- suffix: string (nullable = true)
 | |-- issn: string (nullable = true)
 | |-- other_ids: struct (nullable = true)
 | | |-- DOI: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | |-- pages: string (nullable = true)
 | |-- ref_id: string (nullable = true)
 | |-- title: string (nullable = true)
 | |-- venue: string (nullable = true)
 | |-- volume: string (nullable = true)
 | |-- year: long (nullable = true)
 |-- BIBREF1: struct (nullable = true)
 | |-- authors: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- first: string (nullable = true)
 | | | |-- last: string (nullable = true)
 | | | |-- middle: array (nullable = true)
 | | | | |-- element: string (containsNull = true)
 | | | |-- suffix: string (nullable = true)
 | |-- issn: string (nullable = true)
 | |-- other_ids: struct (nullable = true)
 | | |-- DOI: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- arXiv: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | |-- pages: string (nullable = true)
 | |-- ref_id: string (nullable = true)
 | |-- title: string (nullable = true)
 | |-- venue: string (nullable = true)
 | |-- volume: string (nullable = true)
 | |-- year: long (nullable = true)
 |-- BIBREF10: struct (nullable = true)
 | |-- authors: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- first: string (nullable = true)
 | | | |-- last: string (nullable = true)
 | | | |-- middle: array (nullable = true)
 | | | | |-- element: string (containsNull = true)
 | | | |-- suffix: string (nullable = true)
 | |-- issn: string (nullable = true)
 | |-- other_ids: struct (nullable = true)
 | | |-- DOI: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | |-- pages: string (nullable = true)
 | |-- ref_id: string (nullable = true)
 | |-- title: string (nullable = true)
 | |-- venue: string (nullable = true)
 | |-- volume: string (nullable = true)
 | |-- year: long (nullable = true)
 |-- BIBREF100: struct (nullable = true)
 | |-- authors: array (nullable = true)
 | | |-- eleme

In [0]:
grouped_doc = df.select("paper_id", explode("body_text")\
                .alias("body_text_flat"))\
                .select("paper_id", "body_text_flat.text")\
                .groupby("paper_id")\
                .agg(concat_ws(" ", collect_list("text").alias("text")).alias("body_text"))

# Preprocess Text

In [0]:
%sh

ls /databricks/python3/bin/python3

/databricks/python3/bin/python3

In [0]:
from langid.langid import LanguageIdentifier, model
import string
import re
import spacy
spacy.cli.download("en_core_web_md")
nlp=spacy.load('en_core_web_md')

✔ Download and installation successful 
You can now load the model via spacy.load('en_core_web_md')

In [0]:
import thinc

## Remove None ASCII

In [0]:
def strip_non_ascii(data_str):
    ''' Returns the string without non ASCII characters'''
    stripped = (c for c in data_str if 0 < ord(c) < 127)
    return ''.join(stripped)

## Check Language

In [0]:
def check_lang(data_str):
    identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
    predict_lang = identifier.classify(data_str)

    if predict_lang[1] >= .9:
        language = predict_lang[0]
    else:
        language = predict_lang[0]
    return language

## Remove Stop Words & Lemmatize

In [0]:
# Remove stop words and lemmatize

def clean_text(text):
  if thinc.extra.load_nlp.VECTORS is None:
    nlp = spacy.load('en_core_web_md')
    proc = nlp(text)
    
  try:
    proc = nlp(text)
  except (NameError, UnboundLocalError) as e:
    nlp = spacy.load('en_core_web_md')
    proc = nlp(text)
  else:
    pass
  
  types = ["NOUN", "PROPN", "VERB", "ADJ"]
  
  cleaned = []
  for i, token in enumerate(proc):
    if i == 0:
      cleaned.append(token.lemma_)
    elif token.is_stop is False and token.pos_ in types:
      cleaned.append(token.lemma_)
  
  return " ".join(cleaned)

## Spark UDF

In [0]:
strip_non_ascii_udf = udf(strip_non_ascii, StringType())
check_lang_udf = udf(check_lang, StringType())
clean_udf = udf(clean_text, StringType())

In [0]:
grouped_doc.printSchema()

root
-- paper_id: string (nullable = true)
-- body_text: string (nullable = false)

In [0]:
grouped_doc.show(3)

+--------------------+--------------------+
 paper_id| body_text|
+--------------------+--------------------+
2a21fdd15e07c89c8...|Group testing was...|
0bd443591b5e10934...|December 1, 2019 ...|
2c17a581c10ece6c5...|The coronaviruses...|
+--------------------+--------------------+
only showing top 3 rows

In [0]:
proc_df = grouped_doc.withColumn('no_ascii', strip_non_ascii_udf(grouped_doc['body_text'])).drop("body_text")
proc_df.show(5)

+--------------------+--------------------+
 paper_id| no_ascii|
+--------------------+--------------------+
2a21fdd15e07c89c8...|Group testing was...|
0bd443591b5e10934...|December 1, 2019 ...|
2c17a581c10ece6c5...|The coronaviruses...|
0da2ec30d7dfdef62...|Using the premise...|
0eee9760aee0ca2fd...|The first cluster...|
+--------------------+--------------------+
only showing top 5 rows

In [0]:
proc_df = proc_df.withColumn('lang', check_lang_udf(proc_df['no_ascii']))
proc_df.show(5)

+--------------------+--------------------+----+
 paper_id| no_ascii|lang|
+--------------------+--------------------+----+
2a21fdd15e07c89c8...|Group testing was...| en|
0bd443591b5e10934...|December 1, 2019 ...| en|
2c17a581c10ece6c5...|The coronaviruses...| en|
0da2ec30d7dfdef62...|Using the premise...| en|
0eee9760aee0ca2fd...|The first cluster...| en|
+--------------------+--------------------+----+
only showing top 5 rows

In [0]:
proc_df = proc_df.filter(proc_df['lang'] == "en")

In [0]:
proc_df = proc_df.withColumn('clean', clean_udf(proc_df['no_ascii']))
proc_df.show(5)

+--------------------+--------------------+----+--------------------+
 paper_id| no_ascii|lang| clean|
+--------------------+--------------------+----+--------------------+
2a21fdd15e07c89c8...|Group testing was...| en|group testing des...|
0bd443591b5e10934...|December 1, 2019 ...| en|December Wuhan Ci...|
2c17a581c10ece6c5...|The coronaviruses...| en|the coronaviruse ...|
0da2ec30d7dfdef62...|Using the premise...| en|use premise effec...|
0eee9760aee0ca2fd...|The first cluster...| en|the cluster case ...|
+--------------------+--------------------+----+--------------------+
only showing top 5 rows

# Recommender

In [0]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, NGram
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes, RandomForestClassifier
from pyspark.ml.clustering import LDA
from pyspark.ml.feature import CountVectorizer

# Configure an ML pipeline, which consists of tree stages: tokenizer, hashingTF, and nb.
tokenizer = Tokenizer(inputCol="clean", outputCol="words")
data = tokenizer.transform(proc_df)


In [0]:
data.show(5)

+--------------------+--------------------+----+--------------------+--------------------+
 paper_id| no_ascii|lang| clean| words|
+--------------------+--------------------+----+--------------------+--------------------+
2a21fdd15e07c89c8...|Group testing was...| en|group testing des...|[group, testing, ...|
0bd443591b5e10934...|December 1, 2019 ...| en|December Wuhan Ci...|[december, wuhan,...|
2c17a581c10ece6c5...|The coronaviruses...| en|the coronaviruse ...|[the, coronavirus...|
0da2ec30d7dfdef62...|Using the premise...| en|use premise effec...|[use, premise, ef...|
0eee9760aee0ca2fd...|The first cluster...| en|the cluster case ...|[the, cluster, ca...|
+--------------------+--------------------+----+--------------------+--------------------+
only showing top 5 rows

In [0]:
ngram = NGram(n=2, inputCol='words', outputCol='ngrams')
ngram_df = ngram.transform(data)

In [0]:
hashing = HashingTF(numFeatures=10000, inputCol="ngrams", outputCol="vectors")
hash_df = hashing.transform(ngram_df)

In [0]:
hash_df.show(5)

+--------------------+--------------------+----+--------------------+--------------------+--------------------+--------------------+
 paper_id| no_ascii|lang| clean| words| ngrams| vectors|
+--------------------+--------------------+----+--------------------+--------------------+--------------------+--------------------+
2a21fdd15e07c89c8...|Group testing was...| en|group testing des...|[group, testing, ...|[group testing, t...|(10000,[2,3,4,10,...|
0bd443591b5e10934...|December 1, 2019 ...| en|December Wuhan Ci...|[december, wuhan,...|[december wuhan, ...|(10000,[2,3,22,29...|
2c17a581c10ece6c5...|The coronaviruses...| en|the coronaviruse ...|[the, coronavirus...|[the coronaviruse...|(10000,[1,12,17,1...|
0da2ec30d7dfdef62...|Using the premise...| en|use premise effec...|[use, premise, ef...|[use premise, pre...|(10000,[3,4,11,24...|
0eee9760aee0ca2fd...|The first cluster...| en|the cluster case ...|[the, cluster, ca...|[the cluster, clu...|(10000,[30,36,45,...|
+--------------------+--------------------+----+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows

In [0]:
final_model = hash_df.select('paper_id', 'ngrams', col('vectors').alias('features'))
final_model.persist()
# final_model.show(5)

Out[35]: DataFrame[paper_id: string, ngrams: array<string>, features: vector]

# Jaccard Distance with MinHash LSH

In [0]:
from pyspark.ml.feature import MinHashLSH

In [0]:
mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=3)
model = mh.fit(final_model)

In [0]:
model.transform(final_model).show(3)

+--------------------+--------------------+--------------------+--------------------+
 paper_id| ngrams| features| hashes|
+--------------------+--------------------+--------------------+--------------------+
2a21fdd15e07c89c8...|[group testing, t...|(10000,[2,3,4,10,...|[[1022398.0], [52...|
0bd443591b5e10934...|[december wuhan, ...|(10000,[2,3,22,29...|[[9408.0], [37232...|
2c17a581c10ece6c5...|[the coronaviruse...|(10000,[1,12,17,1...|[[515903.0], [200...|
+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows

In [0]:
final_model.head().features

Out[38]: SparseVector(10000, {2: 1.0, 3: 1.0, 4: 1.0, 10: 3.0, 17: 1.0, 31: 1.0, 39: 1.0, 49: 1.0, 50: 5.0, 51: 1.0, 61: 1.0, 63: 1.0, 79: 2.0, 80: 1.0, 86: 1.0, 90: 1.0, 94: 1.0, 97: 1.0, 99: 1.0, 117: 1.0, 130: 1.0, 136: 1.0, 160: 2.0, 174: 1.0, 181: 1.0, 186: 1.0, 189: 1.0, 196: 1.0, 206: 1.0, 223: 1.0, 224: 3.0, 233: 14.0, 244: 2.0, 245: 1.0, 252: 1.0, 254: 1.0, 258: 1.0, 264: 1.0, 277: 2.0, 281: 1.0, 284: 1.0, 285: 1.0, 287: 1.0, 291: 1.0, 311: 1.0, 315: 1.0, 322: 1.0, 330: 1.0, 333: 1.0, 338: 1.0, 354: 1.0, 362: 1.0, 371: 1.0, 385: 1.0, 386: 2.0, 393: 1.0, 395: 1.0, 406: 1.0, 412: 1.0, 413: 1.0, 415: 1.0, 428: 1.0, 432: 1.0, 441: 1.0, 446: 1.0, 458: 3.0, 469: 1.0, 473: 1.0, 476: 1.0, 491: 1.0, 495: 1.0, 502: 1.0, 503: 1.0, 516: 1.0, 519: 1.0, 533: 1.0, 537: 1.0, 542: 1.0, 544: 1.0, 553: 2.0, 567: 1.0, 570: 1.0, 578: 1.0, 591: 1.0, 592: 1.0, 593: 1.0, 598: 1.0, 602: 1.0, 603: 1.0, 608: 1.0, 614: 1.0, 620: 1.0, 629: 2.0, 631: 1.0, 633: 1.0, 641: 1.0, 647: 1.0, 648: 1.0, 650: 1.0, 652: 1.0, 660: 1.0, 662: 1.0, 672: 1.0, 676: 1.0, 680: 1.0, 681: 1.0, 689: 2.0, 696: 1.0, 706: 1.0, 711: 3.0, 713: 1.0, 736: 2.0, 745: 1.0, 752: 1.0, 764: 2.0, 769: 1.0, 771: 1.0, 774: 1.0, 776: 1.0, 780: 1.0, 782: 1.0, 787: 1.0, 807: 1.0, 815: 1.0, 818: 1.0, 823: 3.0, 825: 1.0, 835: 1.0, 842: 1.0, 846: 1.0, 847: 1.0, 852: 1.0, 865: 1.0, 875: 1.0, 878: 2.0, 882: 1.0, 885: 1.0, 918: 1.0, 931: 1.0, 935: 1.0, 944: 1.0, 955: 1.0, 957: 1.0, 972: 1.0, 992: 2.0, 994: 1.0, 995: 1.0, 996: 1.0, 998: 1.0, 1000: 1.0, 1012: 1.0, 1014: 1.0, 1016: 1.0, 1019: 1.0, 1026: 2.0, 1049: 1.0, 1052: 1.0, 1056: 1.0, 1075: 1.0, 1082: 1.0, 1083: 1.0, 1089: 1.0, 1105: 1.0, 1108: 1.0, 1111: 1.0, 1114: 1.0, 1115: 1.0, 1140: 1.0, 1142: 1.0, 1148: 1.0, 1152: 1.0, 1154: 1.0, 1167: 1.0, 1207: 1.0, 1210: 2.0, 1221: 5.0, 1240: 2.0, 1243: 1.0, 1245: 1.0, 1262: 3.0, 1263: 2.0, 1266: 1.0, 1271: 1.0, 1319: 1.0, 1335: 1.0, 1338: 1.0, 1345: 1.0, 1346: 1.0, 1365: 2.0, 1367: 1.0, 1373: 1.0, 1376: 1.0, 1383: 2.0, 1384: 1.0, 1404: 2.0, 1405: 1.0, 1415: 2.0, 1418: 1.0, 1421: 1.0, 1425: 1.0, 1427: 1.0, 1433: 2.0, 1439: 1.0, 1446: 1.0, 1455: 1.0, 1474: 1.0, 1477: 1.0, 1494: 1.0, 1505: 1.0, 1514: 1.0, 1516: 1.0, 1530: 1.0, 1531: 4.0, 1553: 1.0, 1554: 1.0, 1556: 2.0, 1565: 1.0, 1567: 2.0, 1569: 1.0, 1571: 2.0, 1573: 1.0, 1586: 1.0, 1593: 1.0, 1596: 1.0, 1600: 1.0, 1605: 1.0, 1623: 1.0, 1628: 1.0, 1630: 2.0, 1631: 1.0, 1640: 1.0, 1642: 1.0, 1649: 1.0, 1652: 1.0, 1653: 1.0, 1657: 1.0, 1662: 1.0, 1664: 3.0, 1712: 6.0, 1721: 1.0, 1730: 1.0, 1736: 1.0, 1738: 1.0, 1739: 1.0, 1749: 1.0, 1768: 1.0, 1771: 1.0, 1779: 1.0, 1780: 1.0, 1784: 1.0, 1788: 1.0, 1797: 1.0, 1804: 1.0, 1820: 11.0, 1826: 4.0, 1854: 1.0, 1865: 1.0, 1884: 1.0, 1890: 1.0, 1893: 1.0, 1895: 1.0, 1897: 1.0, 1910: 1.0, 1916: 1.0, 1917: 1.0, 1918: 1.0, 1924: 2.0, 1926: 1.0, 1953: 1.0, 1954: 1.0, 1963: 1.0, 1968: 3.0, 1969: 1.0, 1973: 1.0, 1981: 2.0, 1984: 1.0, 1988: 6.0, 2006: 1.0, 2018: 1.0, 2027: 1.0, 2036: 1.0, 2038: 1.0, 2045: 1.0, 2049: 1.0, 2063: 1.0, 2069: 1.0, 2072: 1.0, 2076: 1.0, 2079: 1.0, 2094: 1.0, 2096: 1.0, 2098: 1.0, 2100: 1.0, 2105: 5.0, 2116: 1.0, 2120: 2.0, 2134: 1.0, 2136: 1.0, 2143: 2.0, 2165: 1.0, 2166: 1.0, 2170: 1.0, 2183: 1.0, 2187: 1.0, 2189: 1.0, 2195: 4.0, 2211: 1.0, 2215: 1.0, 2221: 2.0, 2224: 2.0, 2229: 1.0, 2230: 1.0, 2231: 1.0, 2238: 2.0, 2247: 1.0, 2249: 1.0, 2253: 1.0, 2258: 2.0, 2259: 1.0, 2264: 5.0, 2267: 1.0, 2268: 2.0, 2286: 1.0, 2287: 1.0, 2291: 1.0, 2311: 1.0, 2315: 2.0, 2335: 1.0, 2343: 5.0, 2344: 1.0, 2352: 6.0, 2354: 1.0, 2356: 1.0, 2382: 1.0, 2386: 2.0, 2387: 1.0, 2391: 4.0, 2394: 2.0, 2418: 1.0, 2441: 1.0, 2443: 1.0, 2445: 1.0, 2447: 1.0, 2455: 2.0, 2468: 1.0, 2477: 1.0, 2494: 2.0, 2508: 1.0, 2513: 1.0, 2515: 1.0, 2527: 1.0, 2531: 1.0, 2546: 1.0, 2553: 1.0, 2554: 2.0, 2566: 1.0, 2572: 1.0, 2575: 10.0, 2578: 1.0, 2628: 1.0, 2634: 1.0, 2647: 1.0, 2653: 1.0, 2660: 1.0, 2667: 1.0, 2671: 1.0, 2673: 1.0, 2690: 2.0, 2692: 1.0, 2712: 1.0, 2713: 1.0, 2727: 1.0, 2730: 4.0, 2735: 1.0, 2747: 1.0, 2752: 1.0, 27

Let's now try to use the model to find similar articles

In [0]:
test_df = spark.read.json('/FileStore/tables/CORD19/test.json', multiLine=True)
grouped_test = test_df.select("paper_id", explode("body_text")\
                      .alias("body_text_flat"))\
                      .select("paper_id", "body_text_flat.text")\
                      .groupby("paper_id")\
                      .agg(concat_ws(" ", collect_list("text").alias("text")).alias("body_text"))

In [0]:
grouped_test.show()

+--------------------+--------------------+
 paper_id| body_text|
+--------------------+--------------------+
bf0bd7e3007cf87a4...|Human respiratory...|
+--------------------+--------------------+

In [0]:
test_proc_df = grouped_test.withColumn('no_ascii', strip_non_ascii_udf(grouped_test['body_text']))
test_proc_df = test_proc_df.withColumn('clean', clean_udf(test_proc_df['no_ascii']))
test_proc_df.show()

+--------------------+--------------------+--------------------+--------------------+
 paper_id| body_text| no_ascii| clean|
+--------------------+--------------------+--------------------+--------------------+
bf0bd7e3007cf87a4...|Human respiratory...|Human respiratory...|human respiratory...|
+--------------------+--------------------+--------------------+--------------------+

In [0]:
test_data = tokenizer.transform(test_proc_df)
test_ngram_df = ngram.transform(test_data)
test_hash_df = hashing.transform(test_ngram_df)
test_final = test_hash_df.select('paper_id', 'ngrams', col('vectors').alias('features'))

In [0]:
test_final.show()

+--------------------+--------------------+--------------------+
 paper_id| ngrams| features|
+--------------------+--------------------+--------------------+
bf0bd7e3007cf87a4...|[human respirator...|(10000,[1,3,6,7,1...|
+--------------------+--------------------+--------------------+

In [0]:
key = test_final.head().features

In [0]:
model.approxNearestNeighbors(final_model, key, 5).show()

+--------------------+--------------------+--------------------+--------------------+------------------+
 paper_id| ngrams| features| hashes| distCol|
+--------------------+--------------------+--------------------+--------------------+------------------+
2b90a9a3d33a1babe...|[2 progress, prog...|(10000,[2,10,12,2...|[[17271.0], [1696...|0.8302634184987127|
1c06378f629bfa887...|[the term, term p...|(10000,[2,3,11,14...|[[9408.0], [44598...| 0.843225348782218|
1f9d3f9a1a0e8db6a...|[of type, type ch...|(10000,[2,3,6,8,1...|[[515903.0], [840...|0.8771929824561404|
1d882d90109fb4496...|[. pkdl, pkdl rec...|(10000,[5,8,9,12,...|[[1022398.0], [52...|0.8845750262329486|
2c17a581c10ece6c5...|[the coronaviruse...|(10000,[1,12,17,1...|[[515903.0], [200...|0.8886807599678886|
+--------------------+--------------------+--------------------+--------------------+------------------+